# VAL_DISCRETE

## Overview
The VAL_DISCRETE function selects a value from a list based on a given discrete probability distribution. This is essential for simulations, random sampling, and business scenarios where outcomes are determined by weighted probabilities. 

A discrete probability distribution assigns a probability $p_i$ to each possible outcome $x_i$ such that

```math
\sum_{i=1}^n p_i = 1 \quad \text{and} \quad 0 \leq p_i \leq 1
```

The function implements random selection by drawing a single value $x_k$ from the set $\{x_1, x_2, ..., x_n\}$, where the probability of selecting $x_k$ is $p_k$. This is equivalent to sampling from a categorical distribution:

```math
P(X = x_k) = p_k
```

This approach is widely used in Monte Carlo simulations, scenario analysis, and probabilistic modeling, enabling users to automate random selection in Excel based on business-defined likelihoods.

## Usage
To use the `VAL_DISCRETE` function in Excel, enter it as a formula in a cell, specifying the values and their associated probabilities:

```excel
=VAL_DISCRETE(values, distribution)
```

## Arguments
| Argument     | Type     | Required | Description                                              | Example                  |
|--------------|----------|----------|----------------------------------------------------------|--------------------------|
| values       | 2D list  | Yes      | List of possible values to select from                   | [["Retail", "Wholesale", "Online"]] |
| distribution | 2D list  | Yes      | List of probabilities (must sum to 1)                    | [[0.6, 0.3, 0.1]]        |

## Returns
| Returns | Type   | Description                                 | Example   |
|---------|--------|---------------------------------------------|-----------|
| Result  | Scalar | The value selected according to the weights. | "Retail"  |

## Limitations
- The length of `values` and `distribution` must match.
- Probabilities in `distribution` must sum to 1 (within floating point tolerance).
- If `values` or `distribution` are empty, returns `None`.
- If lengths do not match, returns `None`.
- If probabilities do not sum to 1 (±0.01), returns `None`.
- Handles both numbers and strings as values.
- Returns a single value per call (not an array of samples).
- Only supports numeric or string values.

## Benefits
- Enables Monte Carlo simulations and scenario analysis in Excel.
- Automates random selection based on business-defined likelihoods.
- More flexible and accurate than manual randomization in Excel.
- Supports both numeric and string values for realistic business scenarios.

## Examples

### Simulate Customer Type
Select a customer type based on the following values and probabilities:

|   | A      | B         | C      |
|---|--------|-----------|--------|
| 1 | Retail | Wholesale | Online |
| 2 | 0.6    | 0.3       | 0.1    |

Arguments:
- values: [["Retail", "Wholesale", "Online"]]
- distribution: [[0.6, 0.3, 0.1]]

```excel
=VAL_DISCRETE(A1:C1, A2:C2)
```
*Returns "Retail" 60% of the time, "Wholesale" 30% of the time, "Online" 10% of the time.*

### Select Project Outcome
Select a project outcome based on the following values and probabilities:

|   | A        | B        |
|---|----------|----------|
| 1 | Success  | Failure  |
| 2 | 0.8      | 0.2      |

Arguments:
- values: [["Success", "Failure"]]
- distribution: [[0.8, 0.2]]

```excel
=VAL_DISCRETE(A1:B1, A2:B2)
```
*Returns "Success" 80% of the time, "Failure" 20% of the time.*

In [ ]:
import random
from typing import List, Any, Optional

def val_discrete(values: List[List[Any]], distribution: List[List[float]]) -> Optional[Any]:
    """
    Select a value from a list based on a discrete probability distribution.

    Args:
        values (list of lists): 2D list of possible values to select from.
        distribution (list of lists): 2D list of probabilities (must sum to 1).

    Returns:
        The value selected according to the weights, or None if input is invalid.
    """
    # Input validation
    if not values or not distribution:
        return None
    if not isinstance(values, list) or not isinstance(distribution, list):
        return None
    if not values or not values[0] or not distribution or not distribution[0]:
        return None
    if len(values[0]) != len(distribution[0]):
        return None
    weights = distribution[0]
    # Check if all weights are numbers and sum to 1 (±0.01)
    try:
        total = sum(float(w) for w in weights)
        if abs(total - 1.0) > 0.01:
            return None
    except Exception:
        return None
    # Use random.choices to select
    try:
        return random.choices(values[0], weights=weights)[0]
    except Exception:
        return None

In [ ]:
import ipytest
ipytest.autoconfig()

def test_demo_customer_type():
    values = [["Retail", "Wholesale", "Online"]]
    distribution = [[0.6, 0.3, 0.1]]
    result = val_discrete(values, distribution)
    assert result in ["Retail", "Wholesale", "Online"]

def test_demo_project_outcome():
    values = [["Success", "Failure"]]
    distribution = [[0.8, 0.2]]
    result = val_discrete(values, distribution)
    assert result in ["Success", "Failure"]

def test_numeric_values():
    values = [[1, 2, 3]]
    distribution = [[0.2, 0.5, 0.3]]
    result = val_discrete(values, distribution)
    assert result in [1, 2, 3]

def test_probabilities_not_sum_to_1():
    values = [["A", "B"]]
    distribution = [[0.7, 0.7]]
    result = val_discrete(values, distribution)
    assert result is None

def test_empty_input():
    values = []
    distribution = []
    result = val_discrete(values, distribution)
    assert result is None

def test_mismatched_lengths():
    values = [[1, 2]]
    distribution = [[0.5]]
    result = val_discrete(values, distribution)
    assert result is None

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

demo_cases = [
[    [["Retail", "Wholesale", "Online"]], [[0.6, 0.3, 0.1]]],
    [    [["Success", "Failure"]], [[0.8, 0.2]]],
    [    [[1, 2, 3]], [[0.2, 0.5, 0.3]]]
]

demo = gr.Interface(
    fn=val_discrete,
    inputs=[
        gr.Dataframe(headers=["Value 1", "Value 2", "Value 3"], label="Values", row_count=1, col_count=3, type="array", value=[["Retail", "Wholesale", "Online"]]),
        gr.Dataframe(headers=["Prob 1", "Prob 2", "Prob 3"], label="Distribution", row_count=1, col_count=3, type="array", value=[[0.6, 0.3, 0.1]]),
    ],
    outputs=gr.Textbox(label="Selected Value"),
    examples=demo_cases,
    description="Select a value from a list based on a discrete probability distribution. The probabilities must sum to 1. See documentation for details and examples.",
    flagging_mode="never",
)
demo.launch()